In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import os
import json
import re
import textstat

In [4]:
import sys
import yaml
import requests

shared_path = '/gpfs/data/majorlab/biasaudit'
sys.path.insert(0, shared_path)

## read config
with open(os.path.join(shared_path, "gpt_config.yml"), "r") as cfg:
    config = yaml.safe_load(cfg.read())

In [5]:
payload = json.dumps({"temperature": 0, 
                      "top_p" : 0.95,
                      "max_tokens": 800, 
                      "messages": [
                          {"role": "system", 
                           "content": "You are a helpful assistant."}, 
                          {"role": "user", 
                           "content": "Select a random number between 1 and 100,000"}] }) 
  
headers = {"apikey": config.get('kong').get('apikey'),
           'Content-Type': 'application/json'} 
url = "https://genai-api.prod1.nyumc.org/gpt-4o/v1.0.0/chat/completions" 
  
res = requests.post(url, headers=headers, data=payload) 

if res.status_code != 200:
    print('error')
elif not res.json().get('choices'):
    print('bad payload')
else:
    if len(res.json().get('choices')) > 1:
        print('more than one result')
    for c in res.json().get('choices'):
        result = c.get('message').get('content')
print(result)

Sure, here is a random number between 1 and 100,000: 47,583.


### Prompts

In [8]:
# All metrics together
prompt_template = """
        The following are a discharge summary from hospital admission and a patient-friendly discharge instructions of the same, aimed at a 7th-grade reading level. Please assess if there are any factual errors or significant omissions in the patient-friendly discharge instructions that will lead to harm.
        Significant omissions are defined as the instructions lacking information that is necessary for the patient to understand what occurred during admission. It does not need to include the medical history, medication or follow-up instructions. 
        
        {discharge_sum} \n {PF_sum}. 
        
        Please provide feedback in the following format: 
        {
            "Accuracy" : Correct / nearly all correct / more correct than incorrect / equally correct and incorrect / more incorrect than correct / completely correct
            "Accuracy explanation" : "Your feedback here",
            "Completeness": Complete / nearly all complete / more complete than incomplete / equally complete and incomplete / more incomplete than complete / completely incomplete,
            "Completeness explanation" : "Your feedback here",
            "Risk of harm" : No potential for harm / Potential for emotional distress or inconvenience (mild and transient anxiety or pain or physical discomfort) / Potential for requiring additional treatment / Potential for temporary harm (bodily or psychological injury, but likely not permanent) / Potential for permanent harm (lifelong bodily or psychological injury or increased susceptibility to disease) / Potential for lifelong bodily or psychological injury or disfigurement / Potential for severe permanent harm  / Potential for death   
            "Harm explanation" : "Your feedback here"
        },
        
        Please be succinct."""

#### Accuracy

In [7]:
accuracy_prompt = """
        The following are a discharge summary from hospital admission and a patient-friendly discharge instructions of the same, aimed at a 7th-grade reading level. 
        Please assess if there are any factual errors in the patient-friendly discharge instructions based on the summary, that will lead to harm. This includes misrepresentation of the original summary or introduction of new, unsubstantiated information. 
        
        {discharge_sum} \n {PF_sum}. 
        
        Please provide feedback in the following format: 
        {
            "Accuracy" : Correct / nearly all correct / more correct than incorrect / equally correct and incorrect / more incorrect than correct / completely correct
            "Accuracy explanation" : "Your feedback here",
        }
        
        Please be succinct."""

#### Completeness

In [5]:
complete_prompt = """
        The following are a discharge summary from hospital admission and a patient-friendly discharge instructions of the same, aimed at a 7th-grade reading level. Please assess if there are any factual errors or significant omissions in the patient-friendly discharge instructions that will lead to harm.
        Please assess if there are any pieces of information missing in the instructions that are necessary for the patient to understand what occurred during their admission. It does not need to include the medical history, medication or follow-up instructions. 
        
        {discharge_sum} \n {PF_sum}. 

        Please provide feedback in the following format: 
        {
            "Completeness": Complete / nearly all complete / more complete than incomplete / equally complete and incomplete / more incomplete than complete / completely incomplete,
            "Completeness explanation" : "Your feedback here",
        },

        For example:
        Discharge summary: "DISCHARGE NARRATIVE 
        Admit date: 10/6/2024
        Expected Discharge date: 10/24/2024 ‚Äö√Ñ√£Afternoon by 3 PM
        
        Admission Diagnoses: dyspnea on exertion
         
        Discharge Diagnoses/Hospital Problems:
        Principal Problem:
        Acute on chronic diastolic heart failure
        Active Problems:
        Essential hypertension
        Paroxysmal atrial fibrillation
        Congestive heart failure
        Anemia
        Pulmonary hypertension
        Body mass index (BMI) 38.0-38.9, adult
        Chronic atrial fibrillation
        Resolved Problems:
        * No resolved hospital problems. *
         
        Indication for Admission: Dyspnea
         
        History of Present Illness:   
        XXX with PMH HF, HTN, HLD, pAF, pulmonary HTN, CKD, anemia, leg edema who presents with two-three week of worsening BLE edema and worsening dyspnea on exerction. Pt states occasional ETOH use, former tobacco use. Occasional NSAID use, but stopped d/t blood thinner. 
         
        Last INR 2.9 on 9/12. States she takes warfarin 10mg every night and but she said 3-4 days of the week she takes 5mg. Reports to RN that "her doctor tells her how many to miss not everyday". 
         
        Fall recently, did not have LOC, she was able to sit down. Also reports difficulty with taking meds everyday.
        | |   
        ---|---|---  
        Hospital Course:
        In the ED afebrile, HR 101, other VSS, on RA. Labs notable for: negative lactate, K 4.6, CL 109, CO2 19, Cr 1.49, BUN 22, AST 36, BNP 299, trop 15, no leukocytosis, H/H 5/19, plt 239
         
        CXR: Stable large cardiac silhouette with tortuous aorta. No focal consolidation, pleural effusion or discernible pneumothorax. Visualized osseous structures intact.
         
        Received: 80 mg lasix. Pt admitted to General Medicine for further management of anemia and volume overload.
        Cardiology consulted for HF exacerbation.
        ENT was consulted 2/2 to imaging revealing left submandibular sialadenitis, due to a 1.3x0.7cm stone at the hilum of the left submandibular gland, otherwise uncomplicated. 
         
        ‚Äö√Ñ√£Plan:
        \- Discharge to Skilled nursing facility
        \- Complete 7 day course of Augmentin for salivary gland infection"
        Patient-friendly discharge: "What brought me to the hospital? You came to the hospital because you were having trouble breathing, especially when you were moving around. Why was I hospitalized? You were hospitalized because of acute on chronic diastolic heart failure. This means your heart was not filling with blood properly, which was causing problems for a while and got worse recently. What happened in the hospital? The doctors did many tests. They found that you had too much fluid in your body and low red blood cells. They gave you a medicine called Lasix to help your body get rid of extra fluid. They also found a stone in your salivary gland, which is a small organ in your mouth that makes saliva. You had a special test called a video capsule endoscopy. This test found some bleeding and small blood vessels in your small intestine. What should I know after leaving the hospital? You will go to a skilled nursing facility for more care. You will take a medicine called Augmentin for 7 days to treat the infection in your salivary gland."

        Assessment:
        {
            "Completeness": Nearly all complete,
            "Completeness explanation" : "Summary covers main sailent events of admission",
        }
        
        Please be succinct.

"""

In [17]:
complete_prompt_zero = """
        The following are a discharge summary from hospital admission and a patient-friendly discharge instructions of the same, aimed at a 7th-grade reading level. Please assess if there are any factual errors or significant omissions in the patient-friendly discharge instructions that will lead to harm.
        Please assess if there are any pieces of information missing in the instructions that are necessary for the patient to understand what occurred during their admission. It does not need to include the medical history, medication or follow-up instructions. 
        
        {discharge_sum} \n {PF_sum}. 

        Please provide feedback in the following format: 
        {
            "Completeness": Complete / nearly all complete / more complete than incomplete / equally complete and incomplete / more incomplete than complete / completely incomplete,
            "Completeness explanation" : "Your feedback here",
        }
        
        Please be succinct.

"""

In [9]:
harm_prompt = """
        The following are a discharge summary from hospital admission and a patient-friendly discharge instructions of the same, aimed at a 7th-grade reading level. Please assess if there are any factual errors or significant omissions in the patient-friendly discharge instructions that will lead to harm.
        Significant omissions are defined as the instructions lacking information that is necessary for the patient to understand what occurred during admission. It does not need to include the medical history, medication or follow-up instructions. 
        
        {discharge_sum} \n {PF_sum}. 
        
        Please provide feedback in the following format: 
        {
            "Risk of harm" : No potential for harm / Potential for emotional distress or inconvenience (mild and transient anxiety or pain or physical discomfort) / Potential for requiring additional treatment / Potential for temporary harm (bodily or psychological injury, but likely not permanent) / Potential for permanent harm (lifelong bodily or psychological injury or increased susceptibility to disease) / Potential for lifelong bodily or psychological injury or disfigurement / Potential for severe permanent harm  / Potential for death   
            "Harm explanation" : "Your feedback here"
        },
        
        Please be succinct.

"""

#### Functions

In [16]:
### Updating the gpt function call

def request_gpt(content, config, settings=None):
    payload = { 
        "temperature": 0, 
        "top_p" : 0.95,
        "max_tokens": 800, 
        "messages": content
    }
    if settings: # i.e. not empty or default
        payload.update(settings) # override above defaults with one or more specified
    
    headers = { 
      'apikey': config.get('kong').get('apikey'), 
      'Content-Type': 'application/json' 
    }
    url = "https://genai-api.prod1.nyumc.org/gpt-4o/v1.0.0/chat/completions" 

    res = requests.post(url, headers=headers, data=json.dumps(payload)) 
    response = res.json()

    return response

In [9]:
test_sum = pd.read_csv("20250214_checked3.csv")
test_sum = test_sum.to_dict(orient="records")

In [10]:
dc_test = test_sum[2]['Original_sum']
pf_test = test_sum[2]['PF_sum']

In [14]:
def process_output(text):
    output = text['choices'][0]['message']['content']
    return output

#### Calling judge

In [104]:
# Update this with prompt

def judge_test(discharge_sum, PF_sum):
    # Format the prompt dynamically
    # replace prompt_template with desired prompt
    prompt = prompt_rubric_acc_one.replace("{discharge_sum}", discharge_sum).replace("{PF_sum}", PF_sum)

    # Create message structure
    temp = [{'role': 'user', 'content': prompt}]
    
    # Call the GPT function
    gpt_result = request_gpt(temp, config)

    if not gpt_result.get('choices'):
        print('Bad payload')
        return None

    return process_output(gpt_result)



In [105]:
judge_batch = []
for i in range(0, len(test_sum)): 
    dc_test = test_sum[i]['Original_sum']
    pf_test = test_sum[i]['PF_sum']
    test_i = judge_test(dc_test, pf_test)
    judge_batch.append(test_i)

In [ ]:
parsed_data = [json.loads(item) for item in judge_batch if item is not None]

In [113]:
import ast
import json

parsed_data = []

for idx, item in enumerate(judge_batch):
    if not item or not isinstance(item, str):
        continue

    try:
        # Step 0: Remove Markdown code block formatting if present
        item = item.strip()
        if item.startswith("```") and item.endswith("```"):
            # Remove the triple backticks and optional 'json'
            item = "\n".join(item.split("\n")[1:-1]).strip()

        # Step 1: Use ast.literal_eval to turn string-wrapped string into real string/dict
        result = ast.literal_eval(item)

        # Step 2: If still a string, try to JSON-parse it
        if isinstance(result, str):
            result = json.loads(result)

        parsed_data.append(result)

    except Exception as e:
        print(f"❌ Error at index {idx}:\n{e}\n{repr(item)}\n")



In [ ]:
for idx, item in enumerate(judge_batch):
    try:
        json.loads(item)
    except json.JSONDecodeError as e:
        print(f"⚠️  JSONDecodeError at index {idx}:\n{item}\nError: {e}\n")


In [115]:
judge_batch_df = pd.DataFrame(parsed_data)
judge_batch_df.to_csv("250416_acccriteria_one.csv")

#### Retire

In [24]:
import re
import pandas as pd
import json

def clean_json_string(json_str):
    """Clean and fix the JSON-like string to make it valid JSON"""
    # Remove the leading and trailing single quotes if present
    if json_str.startswith("'") and json_str.endswith("'"):
        json_str = json_str[1:-1]
    
    # First, extract key-value pairs using regex
    pattern = r'"([^"]+)"\s*:\s*(.*?)(?=,\s*"[^"]+":|$)'
    matches = re.findall(pattern, json_str, re.DOTALL)
    
    # Create a new dictionary with properly formatted values
    cleaned_dict = {}
    for key, value in matches:
        # Clean up the value
        value = value.strip()
        if value.endswith(','):
            value = value[:-1].strip()
        
        # Store in the dictionary
        cleaned_dict[key] = value
    
    return cleaned_dict

def convert_to_dataframe(json_list):
    """Convert a list of JSON-like strings to a pandas DataFrame"""
    # Parse each JSON-like string
    parsed_data = []
    for json_str in json_list:
        parsed_dict = clean_json_string(json_str)
        parsed_data.append(parsed_dict)
    
    # Create DataFrame
    df = pd.DataFrame(parsed_data)
    return df

In [27]:
outputs = pd.DataFrame({'all': judge_batch_all,'complete_zero': judge_batch_zero, 'complete_oneshot': judge_batch})
outputs.to_csv("outputs.csv")

In [17]:
test_1 = judge_test(dc_test, pf_test)
print(test_1)

{
    "Accuracy" : more incorrect than correct,
    "Accuracy explanation" : The patient-friendly discharge instructions contain several inaccuracies. The patient was admitted for syncope and new onset atrial fibrillation with rapid ventricular response (RVR), not for SIRS or a UTI. The instructions also incorrectly state that the patient had a UTI, which was not mentioned in the discharge narrative. Additionally, the instructions do not mention the patient's preserved ejection fraction (EF), mild aortic regurgitation (AR), and mild mitral regurgitation (MR) found on the echocardiogram.
    "Completeness": more incomplete than complete,
    "Completeness explanation" : The instructions omit important details such as the patient's preserved EF, mild AR, and mild MR. They also fail to mention the need for follow-up with outpatient pulmonary for the new lung lesion and the need for an MRI for the shoulder pain. These omissions are significant as they are necessary for the patient to under

#### Rubric automation

In [69]:
prompt_rubric_1 = """
        The following are a discharge summary from hospital admission and a patient-friendly discharge instructions of the same, aimed at a 7th-grade reading level. Please assess if there are any factual errors or significant omissions in the patient-friendly discharge instructions that will lead to harm.
        Significant omissions are defined as the instructions lacking information that is necessary for the patient to understand what occurred during admission. It does not need to include the medical history, medication or follow-up instructions. 
        
        {discharge_sum} \n {PF_sum}. 
        
        Please provide feedback based on the following criterion, with the output as valid json:
        {
            Understandability : [Evaluate using PEMAT-U]    
            Completeness : [Complete / nearly all complete / more complete than incomplete / equally complete and incomplete / more incomplete than complete / completely incomplete],
            Completeness explanation : [Your explanation here],
            Accuracy : [Correct / nearly all correct / more correct than incorrect / equally correct and incorrect / more incorrect than correct / completely incorrect],
            Accuracy explanation : [Your explanation here],
            Harm :  [No potential for harm / Potential for emotional distress or inconvenience (mild and transient anxiety or pain or physical discomfort) / Potential for requiring additional treatment / Potential for temporary harm (bodily or psychological injury, but likely not permanent) / Potential for permanent harm (lifelong bodily or psychological injury or increased susceptibility to disease) / Potential for lifelong bodily or psychological injury or disfigurement / Potential for severe permanent harm  / Potential for death],  
            Harm explanation : [Your explanation here],
            Use person-first language: [Yes / No],
            Refer to patients as how they want to be identified : [Yes / No],
            Avoid abbreviations and acronyms : [Yes / No],
            Avoid words which may convey bias or judgment : [Yes / No],
            Descriptions of physical exam limited to objective findings : [Yes / No],
            Empowers patients with encouraging words and clear next steps : [Yes / No], 
            Pay close attention to sensitive topics, including but not limited to sexual history, trauma history, substance use history, mental health or illness : [Yes / No],
            Explanation of patient-centered language : [Your explanation here]
        }

        Please be succinct."""

In [78]:
prompt_rubric_acc = """
        The following are a discharge summary from hospital admission and a patient-friendly discharge instructions of the same, aimed at a 7th-grade reading level. Please assess if there are any factual errors or significant omissions in the patient-friendly discharge instructions that will lead to harm.
        Significant omissions are defined as the instructions lacking information that is necessary for the patient to understand what occurred during admission. It does not need to include the medical history, medication or follow-up instructions. 
        
        {discharge_sum} \n {PF_sum}. 
        
        Please provide feedback based on the following criterion, with the output as valid json:
        {
            Completeness : [Complete / nearly all complete / more complete than incomplete / equally complete and incomplete / more incomplete than complete / completely incomplete],
            Completeness explanation : [Your explanation here],
            Accuracy : [Correct / nearly all correct / more correct than incorrect / equally correct and incorrect / more incorrect than correct / completely incorrect],
            Accuracy explanation : [Your explanation here],
            Harm :  [No potential for harm / Potential for emotional distress or inconvenience (mild and transient anxiety or pain or physical discomfort) / Potential for requiring additional treatment / Potential for temporary harm (bodily or psychological injury, but likely not permanent) / Potential for permanent harm (lifelong bodily or psychological injury or increased susceptibility to disease) / Potential for lifelong bodily or psychological injury or disfigurement / Potential for severe permanent harm  / Potential for death],  
            Harm explanation : [Your explanation here]
        }

        Please be succinct."""

In [102]:
prompt_rubric_acc_one = """
        The following are a discharge summary from hospital admission and a patient-friendly discharge instructions of the same, aimed at a 7th-grade reading level. Please assess if there are any factual errors or significant omissions in the patient-friendly discharge instructions that will lead to harm.
        Significant omissions are defined as the instructions lacking information that is necessary for the patient to understand what occurred during admission. It does not need to include the medical history, medication or follow-up instructions. 
        
        {discharge_sum} \n {PF_sum}. 
        
        Please provide feedback based on the following criterion, with the output as valid json:
        {
            Completeness : [Complete / nearly all complete / more complete than incomplete / equally complete and incomplete / more incomplete than complete / completely incomplete],
            Completeness explanation : [Your explanation here],
            Accuracy : [Correct / nearly all correct / more correct than incorrect / equally correct and incorrect / more incorrect than correct / completely incorrect],
            Accuracy explanation : [Your explanation here],
            Harm :  [No potential for harm / Potential for emotional distress or inconvenience (mild and transient anxiety or pain or physical discomfort) / Potential for requiring additional treatment / Potential for temporary harm (bodily or psychological injury, but likely not permanent) / Potential for permanent harm (lifelong bodily or psychological injury or increased susceptibility to disease) / Potential for lifelong bodily or psychological injury or disfigurement / Potential for severe permanent harm  / Potential for death],  
            Harm explanation : [Your explanation here]
        }

        Here is an example:
            Discharge summary: "DISCHARGE NARRATIVE Admit date: 10/8/2024 Expected Discharge date: 10/14/2024 Attending Authorizing Discharge: Myint Myint Aye, MD
            Admission Diagnoses: Osteomyelitis Discharge Diagnoses/Hospital Problems:
            Principal Problem: Discitis of lumbar region Resolved Problems:
            * No resolved hospital problems. * Indication for Admission: Osteomyelitis requiring IV abx
            History of Present Illness: 69 y.o. female with a history of BRCA1 mutation (self-reported, no genetic testing available) breast cancer diagnosed in 2005 s/p bilateral mastectomies and TAH/BSO in 2006 and treatment with anthracycline/trastuzumab, HFrEF 2/2 chemotherapy-induced cardiomyopathy was weaned off milrinone drip two weeks ago, pAF, HTN, HLD, MRI Lumbar spine in May with findings highly concerning for multifocal discitis/osteomyelitis presented after being repeatedly encouraged by her outpatient physicians to go to the ED. 
            When talking to the patient today, she states that she has minimum pain of her lower back after taking Tylneol. At baseline it is a 2-3/10 on most days, however has pain flare intermittently where she is unable to walk around. States that she took Aleve x2 1 week ago and started to notice blood in her urine, prompting her to stop. Denies fevers, chills, n/v/d. Denies urinary or bowel incontinence. Describes the pain as localized in the lumbar region with extension into the right lower back. No shooting pain involved. 
            Neuro history: The patient first visited the emergency department (ED) on February 17, 2024, reporting moderate pain attributed to musculoskeletal issues, for which she was discharged with Tylenol and cyclobenzaprine. She returned on March 2, needing assistance to walk, and a CT scan revealed chronic degenerative changes in her lumbar spine. After a subsequent visit on March 16, she was admitted for pain management and physical therapy, showing improvement before being discharged. However, an MRI ordered by her primary care physician on May 18 indicated significant concerns, revealing marked marrow replacement and edema in the vertebral bodies L1-L5, along with findings suggestive of multifocal discitis or osteomyelitis, and potential abscess formation contributing to severe thecal sac narrowing.    
            Cardiac Hx:  She was diagnosed with HF in June 2021 iso new afib. ECHO was done in 2021 which showed EF of 51%. In June 2023 sh presented to NYU BK for worsening SOB and LE edema noted to have a low output state and started on dobutamine. Underwent right heart cath revealing low CO, being then transitioned to milrinone. She has had a PICC line since then for milrinone. Subsequently weaned off in 8/2024 after multiorgan dysfunction and worsening HF symptoms. Since then, she has been on torsemide 20mg daily. 
            ED Course: While in the ED, the patient was hemodynamically stable. Pertinent lab and imaging findings are included below. Significant ED interventions included ceftriaxone 2,000mg and removal of PICC from the right upper extremity. The patient is to be admitted to the medicine service for further management of lumbar discitis/osteomyelitis.
            Hospital Course: She underwent further imaging with MRI which showed findings at L1-2 compatible with discitis/OM and epidural phlegmon. ECHO done to r/o endocarditis, which was unremarkable. Neurology and neurosurgery evaluated the patient with no acute interventions. Bone biopsy was ultimately deferred due risk/benefit analysis after discussion with ID and neuro IR. She was discharged wif IV abx Daptomycin 8 mg/kg q24h, Ceftriaxone 2g daily through PICC for 6 weeks, with estimated stop date of 11/21/24.
            Hospital course c/b AKI likely due to dehydration, which resolved. She was discharged with IV abx and close ID follow up.
            Procedures:   PICC
            Disposition: Home/ Self Care"
            
            Patient friendly discharge summary: "What brought me to the hospital? You came to the hospital because of pain in your lower back. Why was I hospitalized? You were hospitalized because of osteomyelitis and discitis of the lumbar region. Osteomyelitis means an infection in your bone. Discitis means an infection in the discs in your spine. These infections needed strong medicine given through a needle in your vein. What happened in the hospital? The doctors did a special scan called an MRI. They found signs of infection in your spine. They also found a phlegmon, which is a pocket of infection. You did not need surgery. The doctors gave you strong medicine through a needle in your vein to fight the infection. You also had a small procedure to put a PICC line. A PICC line is a long, thin tube that goes into your vein. It helps give medicine. What should I know after leaving the hospital? You will continue the strong medicine at home through the PICC line for 6 weeks. You will need to see an infection doctor closely."

        {
            "Completeness" : "Nearly all complete",
            "Completeness explanation" : "Minor omission of absence of neurology, omission of AKI but no significant omissions",
            "Accuracy" : "Correct",
            "Accuracy explanation" : "No evidence of inaccuracy",
            "Harm" :  "Potential for emotional distress or inconvenience (mild and transient anxiety or pain or physical discomfort)",  
            "Harm explanation" : "Risk of small amount of distress from above omissions, but no cause of significant harm"
        
        }

        Please be succinct."""

In [92]:
prompt_rubric_pf = """
        The following are a discharge summary from hospital admission and a patient-friendly discharge instructions of the same, aimed at a 7th-grade reading level. Please assess if there are any factual errors or significant omissions in the patient-friendly discharge instructions that will lead to harm.
        Significant omissions are defined as the instructions lacking information that is necessary for the patient to understand what occurred during admission. It does not need to include the medical history, medication or follow-up instructions. 
        
        {discharge_sum} \n {PF_sum}. 
        
        Please provide feedback based on the following criterion, with the output as valid json:
        {
            Use person-first language: [Yes / No],
            Refer to patients as how they want to be identified : [Yes / No],
            Avoid abbreviations and acronyms : [Yes / No],
            Avoid words which may convey bias or judgment : [Yes / No],
            Descriptions of physical exam limited to objective findings : [Yes / No],
            Empowers patients with encouraging words and clear next steps : [Yes / No], 
            Pay close attention to sensitive topics, including but not limited to sexual history, trauma history, substance use history, mental health or illness : [Yes / No],
            Explanation of patient-centered language : [Your explanation here]
        }

        Please be succinct."""